In [1]:
#!/usr/bin/python3
"""Testing On Segmentation Task."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
import sys
import math
import h5py
import argparse
import importlib
import data_utils
import numpy as np
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm
# import pykitti


In [2]:
batch_size = 1
sample_num = 1

In [2]:
__predicate = "argo_1"
_file_loc  = "/home/kartik/DL_model/PointCNN/"
_file_loc_raw_data  = _file_loc + "test_data/data/"
__dataset = "/home/kartik/DL_model/PointCNN/data/Argo_h5_GR_scaled_depth/train/val.txt"
__dataset_final_result = _file_loc+"test_data/results/"
__model = "/home/kartik/DL_model/PointCNN/outfolder_argo_gr2/ckpts/"+"iter-76444.data-00000-of-00001"
pred_path = _file_loc+"/pointcnn-dataset/"+__predicate+"/val"
pred_path = _file_loc+"/outfolder_test_"+__predicate+"/val"


In [68]:
def data_preprocessing(filename , max_point_num):
    batch_size = 2048
    block_size = 1000
    grid_size = 0.25
        
    # data_filename = FLAGS.image_filename
    ROOT_DIR = os.getcwd()

    # Directory to save logs and trained model
    MODEL_DIR = os.path.join(ROOT_DIR, "logs_test")

    # Directory to get binary and image data
    PARENT_DIR = os.path.abspath(os.path.join(ROOT_DIR, os.pardir))
    DATA_DIR = os.path.join(PARENT_DIR, "test_dataset")
    
    data = np.zeros((batch_size, max_point_num, 4))
    data_num = np.zeros((batch_size), dtype=np.int32)
    label = np.zeros((batch_size), dtype=np.int32)
    label_seg = np.zeros((batch_size, max_point_num), dtype=np.int32)
    indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)
    
#     xyzi = np.fromfile(
#         os.path.join(filename),
#         dtype=np.float32).reshape((-1, 4))
    xyzi = np.load(source_files[0]).reshape((-1, 4))
    pts = xyzi[:, :3]
    depth = np.linalg.norm(xyzi, 2, axis=1)
    xyzi = np.hstack((pts, depth[:, np.newaxis]))
    
    indices_for_prediction = np.arange(xyzi.shape[0]) #(xyzi[:,0] >= -5 ).nonzero()[0]
    xyzif =xyzi #= xyzi[xyzi[:,0] >= -5 ] 
    
    all_label_pred = np.zeros((xyzi.shape[0]),dtype=int)
    label_length = xyzif.shape[0]
    xyz =xyzif[:,0:3]
    
    i =( xyzif[:,3:4] / (np.max(xyzif[:,3:4].flatten()) + 1e-10)  ) 
            
    xyz_min = np.amin(xyz, axis=0, keepdims=True)
    xyz_max = np.amax(xyz, axis=0, keepdims=True)
    block_size = (2 * (xyz_max[0, 0] - xyz_min[0, 0]), 2 * (xyz_max[0, 1] - xyz_min[0, 1]) ,  2 * (xyz_max[0, -1] - xyz_min[0, -1]))
    
    xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)

    #print('{}-Collecting points belong to each block...'.format(datetime.now(), xyzrcof.shape[0]))
    blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True,
                                                                return_counts=True, axis=0)
    block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
    #print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

    block_to_block_idx_map = dict()
    for block_idx in range(blocks.shape[0]):
        block = (blocks[block_idx][0], blocks[block_idx][1])
        block_to_block_idx_map[(block[0], block[1])] = block_idx

    # merge small blocks into one of their big neighbors
    block_point_count_threshold = max_point_num / 3
    #print("block_point_count_threshold",block_point_count_threshold)
    nbr_block_offsets = [(0, 1), (1, 0), (0, -1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)]
    block_merge_count = 0
    for block_idx in range(blocks.shape[0]):
        if block_point_counts[block_idx] >= block_point_count_threshold:
            #print(block_idx, block_point_counts[block_idx])

            continue


        block = (blocks[block_idx][0], blocks[block_idx][1])
        for x, y in nbr_block_offsets:
            nbr_block = (block[0] + x, block[1] + y)
            if nbr_block not in block_to_block_idx_map:
                continue

            nbr_block_idx = block_to_block_idx_map[nbr_block]
            if block_point_counts[nbr_block_idx] < block_point_count_threshold:
                continue


            #print(block_idx, nbr_block_idx, block_point_counts[nbr_block_idx])

            block_point_indices[nbr_block_idx] = np.concatenate(
                [block_point_indices[nbr_block_idx], block_point_indices[block_idx]], axis=-1)
            block_point_indices[block_idx] = np.array([], dtype=np.int)
            block_merge_count = block_merge_count + 1
            break
    #print('{}-{} of {} blocks are merged.'.format(datetime.now(), block_merge_count, blocks.shape[0]))

    idx_last_non_empty_block = 0
    for block_idx in reversed(range(blocks.shape[0])):
        if block_point_indices[block_idx].shape[0] != 0:
            idx_last_non_empty_block = block_idx
            break

    # uniformly sample each block
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        #print(block_idx, point_indices.shape)
        block_points = xyz[point_indices]
        block_min = np.amin(block_points, axis=0, keepdims=True)
        xyz_grids = np.floor((block_points - block_min) / grid_size).astype(np.int)
        grids, point_grid_indices, grid_point_counts = np.unique(xyz_grids, return_inverse=True,
                                                                 return_counts=True, axis=0)
        grid_point_indices = np.split(np.argsort(point_grid_indices), np.cumsum(grid_point_counts[:-1]))
        grid_point_count_avg = int(np.average(grid_point_counts))
        point_indices_repeated = []
        for grid_idx in range(grids.shape[0]):
            point_indices_in_block = grid_point_indices[grid_idx]
            repeat_num = math.ceil(grid_point_count_avg / point_indices_in_block.shape[0])
            if repeat_num > 1:
                point_indices_in_block = np.repeat(point_indices_in_block, repeat_num)
                np.random.shuffle(point_indices_in_block)
                point_indices_in_block = point_indices_in_block[:grid_point_count_avg]
            point_indices_repeated.extend(list(point_indices[point_indices_in_block]))
        block_point_indices[block_idx] = np.array(point_indices_repeated)
        block_point_counts[block_idx] = len(point_indices_repeated)

    idx = 0
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        block_point_num = point_indices.shape[0]
        block_split_num = int(math.ceil(block_point_num * 1.0 / max_point_num))
        point_num_avg = int(math.ceil(block_point_num * 1.0 / block_split_num))
        point_nums = [point_num_avg] * block_split_num
        point_nums[-1] = block_point_num - (point_num_avg * (block_split_num - 1))
        starts = [0] + list(np.cumsum(point_nums))

        np.random.shuffle(point_indices)
        block_points = xyz[point_indices]


        block_min = np.amin(block_points, axis=0, keepdims=True)
        block_max = np.amax(block_points, axis=0, keepdims=True)
        #block_center = (block_min + block_max) / 2
        #block_center[0][-1] = block_min[0][-1]
        #block_points = block_points - block_center  # align to block bottom center
        x, y, z = np.split(block_points, (1, 2), axis=-1)

        block_xzyrgbi = np.concatenate([x, z, y, i[point_indices]], axis=-1)

        for block_split_idx in range(block_split_num):
            start = starts[block_split_idx]
            point_num = point_nums[block_split_idx]
            #print(block_split_num, block_split_idx, point_num )



            end = start + point_num
            idx_in_batch = idx % batch_size
            data[idx_in_batch, 0:point_num, ...] = block_xzyrgbi[start:end, :]
            data_num[idx_in_batch] = point_num
            indices_split_to_full[idx_in_batch, 0:point_num] = point_indices[start:end]

            #print("indices_split_to_full", idx_in_batch, point_num, indices_split_to_full)

            if  (block_idx == idx_last_non_empty_block and block_split_idx == block_split_num - 1): #Last iteration

                item_num = idx_in_batch + 1
                
            idx = idx + 1
            
    return label_length, data, data_num, indices_split_to_full, item_num, all_label_pred, indices_for_prediction

In [4]:
train_root_folder = "/home/kartik/DL_model/PointCNN/outfolder_argo5"
folder_ckpt = os.path.join(train_root_folder, 'ckpts')
"checkpoint ", folder_ckpt

('checkpoint ', '/home/kartik/DL_model/PointCNN/outfolder_argo5/ckpts')

In [20]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import h5py
import argparse
import numpy as np
from datetime import datetime
from tqdm import tqdm

import data_utils

import numpy as np
from tqdm import tqdm

import importlib
import tensorflow as tf

import glob

In [21]:
max_point_num = 8192

# data_filename = FLAGS.image_filename
ROOT_DIR = os.getcwd()
print(ROOT_DIR)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Directory to get binary and image data
PARENT_DIR = os.path.abspath(os.path.join(ROOT_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, "test_data", "data")
source_files = glob.glob(DATA_DIR+"/*")

/home/kartik/DL_model/PointCNN


In [7]:
args_model = "pointcnn_seg"
args_setting = "kitti3d_x8_2048_fps"
args_repeat_num = 1

model = importlib.import_module(args_model)
setting_path = os.path.join(os.path.join(ROOT_DIR), args_model)
print(setting_path)
sys.path.append(setting_path)
setting = importlib.import_module(args_setting)

sample_num = setting.sample_num
batch_size = args_repeat_num * int ( math.ceil(max_point_num / sample_num) )

######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(batch_size, None, 2), name="indices")
is_training = tf.placeholder(tf.bool, name='is_training')
pts_fts = tf.placeholder(tf.float32, shape=(batch_size, max_point_num, setting.data_dim), name='points')
global_step = tf.Variable(0, trainable=False, name='global_step')
######################################################################

# # Placeholders
# indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
# xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
# rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
# jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
# global_step = tf.Variable(0, trainable=False, name='global_step')
# is_training = tf.placeholder(tf.bool, name='is_training')

# pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
# labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
# labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')


######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if not setting.use_extra_features:
        features_sampled = None
else:
    points_sampled = pts_fts_sampled
    features_sampled = None

net = model.Net(points_sampled, features_sampled, is_training, setting)
seg_probs_op = tf.nn.softmax(net.logits, name='seg_probs')

# for restore model
saver = tf.train.Saver()

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


/home/kartik/DL_model/PointCNN/pointcnn_seg
sampling path  /home/kartik/DL_model/PointCNN/sampling
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2020-01-23 11:08:41.803376-Parameter number: 11510276.


In [78]:
import math
output_location = os.path.join(ROOT_DIR, "test_data", "results")
with tf.Session() as sess:
    
#     sess.run(init_op)
    # Load the model
    print("here")
    latest_ckpt = tf.train.latest_checkpoint("/home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts")
    print(folder_ckpt)
    print(latest_ckpt)
    if latest_ckpt:
        print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
        saver.restore(sess, latest_ckpt)
        print('{}-Checkpoint loaded from {} (Iter {})'.format(
            datetime.now(), latest_ckpt, sess.run(global_step)))
    print("donw")
    indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))
    all_files = source_files
    # The locaton of trh classes ais defined as below
    file_count = 0
    for fname in tqdm(all_files):
        label_length, data, data_num, indices_split_to_full, item_num, all_label_pred, indices_for_prediction = data_preprocessing(fname , max_point_num)

        merged_label_zero = np.zeros((label_length),dtype=int)
        merged_confidence_zero = np.zeros((label_length),dtype=float)
        data =data[0:item_num, ...].astype(np.float32) 
        data_num =data_num[0:item_num, ...] 
        indices_split_to_full = indices_split_to_full[0:item_num]
        batch_num = data.shape[0]

        labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
        confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)
        print(label_length)
#         for batch_idx in range(batch_num):
#             points_batch = data[[batch_idx] * batch_size, ...]
#             point_num = data_num[batch_idx]

#             tile_num = int ( math.ceil((sample_num * batch_size) / point_num) )
#             indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
#             np.random.shuffle(indices_shuffle)
#             indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
#             indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)


#             seg_probs = sess.run([seg_probs_op],
#                                     feed_dict={
#                                         pts_fts: points_batch,
#                                         indices: indices_batch.astype(np.int32),
#                                         is_training: False,
#                                     })
#             probs_2d = np.reshape(seg_probs, (sample_num * batch_size, -1))

#             predictions = [(-1, 0.0)] * point_num
#             for idx in range(sample_num * batch_size):
#                 point_idx = indices_shuffle[idx]
#                 probs = probs_2d[idx, :]
#                 confidence = np.amax(probs)
#                 label = np.argmax(probs)
#                 if confidence > predictions[point_idx][1]:
#                     predictions[point_idx] = [label, confidence]
#             labels_pred[batch_idx, 0:point_num] = np.array([label for label, _ in predictions])
#             confidences_pred[batch_idx, 0:point_num] = np.array([confidence for _, confidence in predictions])
            
            
        labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
        confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)

        #print('{}-{:d} testing batches.'.format(datetime.now(), batch_num))
        for batch_idx in tqdm(range(batch_num)):
            #if batch_idx % 10 == 0:
            #print('{}-Processing {} of {} batches.'.format(datetime.now(), batch_idx, batch_num))
            points_batch = data[[batch_idx] * batch_size, ...]
            point_num = data_num[batch_idx]
            if(point_num <= 10):
                continue 

            tile_num = int(math.ceil((sample_num * batch_size) / point_num))
            indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
            np.random.shuffle(indices_shuffle)
            indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
            indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

            seg_probs = sess.run([seg_probs_op],
                                    feed_dict={
                                        pts_fts: points_batch,
                                        indices: indices_batch,
                                        is_training: False,
                                    })
            probs_2d = np.reshape(seg_probs, (sample_num * batch_size, -1))

            predictions = [(-1, 0.0)] * point_num
            for idx in range(sample_num * batch_size):
                point_idx = indices_shuffle[idx]
                probs = probs_2d[idx, :]
                confidence = np.amax(probs)
                label = np.argmax(probs)
                if confidence > predictions[point_idx][1]:
                    predictions[point_idx] = [label, confidence]
            labels_pred[batch_idx, 0:point_num] = np.array([label for label, _ in predictions])
            confidences_pred[batch_idx, 0:point_num] = np.array([confidence for _, confidence in predictions])


            
        for idx in range(batch_num): #Get highest confidence
            pred = labels_pred[idx].astype(np.int64)
            _indices = indices_split_to_full[idx].astype(np.int64)
            confidence = confidences_pred[idx].astype(np.float32)
            num = data_num[idx].astype(np.int64)

            for i in range(pred.shape[0]):

                 if confidence[i] > 0.8 and confidence[i] > merged_confidence_zero[_indices[i]]:
                    merged_confidence_zero[_indices[i]] = confidence[i]
                    merged_label_zero[_indices[i]] = pred[i]


        all_label_pred[indices_for_prediction] = merged_label_zero
        all_label_pred.tofile(output_location+"/out_{}".format(file_count))
        file_count+=1
        print("saved to ", output_location+"/out_{}".format(file_count))
        break

here
/home/kartik/DL_model/PointCNN/outfolder_argo5/ckpts
/home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946
2020-01-23 15:23:01.968201-Found checkpoint /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946
INFO:tensorflow:Restoring parameters from /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946





  0%|          | 0/167 [00:00<?, ?it/s]

2020-01-23 15:23:02.092267-Checkpoint loaded from /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946 (Iter 141946)
donw






  0%|          | 0/21 [00:00<?, ?it/s]

136322






  5%|▍         | 1/21 [00:00<00:13,  1.48it/s]



 10%|▉         | 2/21 [00:00<00:09,  1.91it/s]



 14%|█▍        | 3/21 [00:01<00:07,  2.40it/s]



 19%|█▉        | 4/21 [00:01<00:05,  2.93it/s]



 24%|██▍       | 5/21 [00:01<00:04,  3.57it/s]



 29%|██▊       | 6/21 [00:01<00:03,  4.05it/s]



 33%|███▎      | 7/21 [00:01<00:03,  4.48it/s]



 38%|███▊      | 8/21 [00:01<00:02,  4.81it/s]



 43%|████▎     | 9/21 [00:01<00:02,  5.09it/s]



 48%|████▊     | 10/21 [00:02<00:02,  5.30it/s]



 52%|█████▏    | 11/21 [00:02<00:01,  5.46it/s]



 57%|█████▋    | 12/21 [00:02<00:01,  5.59it/s]



 62%|██████▏   | 13/21 [00:02<00:01,  5.67it/s]



 67%|██████▋   | 14/21 [00:02<00:01,  5.72it/s]



 71%|███████▏  | 15/21 [00:02<00:00,  6.12it/s]



 76%|███████▌  | 16/21 [00:03<00:00,  6.06it/s]



 81%|████████  | 17/21 [00:03<00:00,  6.01it/s]



 86%|████████▌ | 18/21 [00:03<00:00,  5.96it/s]



 90%|█████████ | 19/21 [00:03<00:00,  6.31it/s]



 95%|█████████▌| 20/21 [00:03<00:00,

saved to  /home/kartik/DL_model/PointCNN/test_data/results/out_1


In [8]:

# _traindata = _file_loc_raw_data + "val/val.txt"
# _traindatacontent = open(_traindata, "r")
# testlists = _traindatacontent.readlines()
# testlists = [int(_d) for _d in testlists]

In [70]:
data.shape

(21, 8192, 4)

In [71]:
data.reshape(-1,4).shape

(172032, 4)

In [72]:
data = np.load(source_files[0])
data.shape

(136322, 4)

In [82]:
pts = data.reshape(-1,4)

In [73]:
all_label_pred.__len__()

136322

In [57]:
cwd = os.getcwd()
loc = os.path.join(cwd, "test_data", "results", "out_0")
labels = np.fromfile(loc, dtype=np.int32)
labels.__len__()

272644

In [63]:
data = np.load(source_files[0])

In [31]:
len(data)

136322

In [74]:
from pyntcloud import PyntCloud
from pyntcloud.io import bin as io_bin
import pandas as pd
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink

In [75]:
def hsv2rgb(h,s,v):
    import colorsys
    return list(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))
        
def hsbcolor(max_expected = 10, sat = 175, bri = 175):
    hue_factor = 255 / max_expected
    rgb_list = [hsv2rgb(((i*hue_factor)%255)/255, sat/255, bri/255) for i in range(max_expected)]
    color_list = np.array(rgb_list, dtype=np.uint8)
    return color_list

In [76]:
# Make seperate colors for segmentations
list0fpointindex = all_label_pred
assert len(new_pts) == len(list0fpointindex) 
N = len(np.unique(list0fpointindex)) # Number of segmentation classes
colorlist = hsbcolor(N) # Color list
labelcolordict = {each:id for id,each in enumerate(np.unique(list0fpointindex))}
classcolorlist = [colorlist[labelcolordict[each]] for each in list0fpointindex]

In [84]:
# Vizualize pointcloud with intensity
new_pts = pd.DataFrame(pts[:, 0:3], columns=['x', 'y', 'z'])
# new_pts["red"]  = 0
# new_pts["green"]  = 0
# new_pts["blue"]  = 0
# new_pts[["red","green", "blue"]] = classcolorlist
# new_pts.loc[list0fpointindex,["red","green", "blue"]] = blue
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(7.816410064697266, 12.029518276453018, 301.9…

### Checking data 

In [12]:
import os
import sys
import math
import random
import shutil
import argparse
import importlib
import data_utils_legacy as data_utils
import numpy as np
import pointfly as pf
import tensorflow as tf
from datetime import datetime
import pandas as pd
from pyntcloud import PyntCloud

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--filelist', '-t', default="data/KITTI/ImageSets/train.txt", help='Path to training set ground truth (.txt)')
parser.add_argument('--filelist_val', '-v', default="data/KITTI/ImageSets/train.txt", help='Path to validation set ground truth (.txt)')
parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
parser.add_argument('--save_folder', '-s', default="/home/kartik/saved/", help='Path to folder for saving check points and summary')
parser.add_argument('--model', '-m', default="pointcnn_seg", help='Model to use')
parser.add_argument('--setting', '-x', default="kitti3d_x8_2048_fps_mod1", help='Setting to use')
parser.add_argument('--epochs', default="1",help='Number of training epochs (default defined in setting)', type=int)
parser.add_argument('--batch_size', default="1", help='Batch size (default defined in setting)', type=int)
# default="64",
parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')
args = parser.parse_args([])

print(args)

Namespace(batch_size=1, epochs=1, filelist='data/KITTI/ImageSets/train.txt', filelist_val='data/KITTI/ImageSets/train.txt', load_ckpt=None, log='log.txt', model='pointcnn_seg', no_code_backup=False, no_timestamp_folder=False, save_folder='/home/kartik/saved/', setting='kitti3d_x8_2048_fps_mod1')


In [3]:
cwd = os.getcwd()
model = importlib.import_module(args.model)
setting_path = os.path.join(cwd, args.model)
sys.path.append(setting_path)
setting = importlib.import_module(args.setting)

num_epochs = args.epochs or setting.num_epochs
batch_size = args.batch_size or setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
label_weights_list = setting.label_weights
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val

In [4]:
points = []
labels = []
point_nums = []
labels_seg = []
indices_split_to_full = []

print('{}-Preparing datasets...'.format(datetime.now()))
# is_list_of_h5_list = data_utils.is_h5_list(args.filelist)
args.filelist = os.path.join(cwd, "data", "Argo_h5_GR_scaled_depth", "train")+"/train.txt"
args.filelist_val = os.path.join(cwd, "data", "Argo_h5_GR_scaled_depth", "train")+"/val.txt"

is_list_of_h5_list = not data_utils.is_h5_list(args.filelist)
if is_list_of_h5_list:
    seg_list = data_utils.load_seg_list(args.filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train = args.filelist
data_train, _, data_num_train, label_train, _ = data_utils.load_seg_Argo(filelist_train)
data_val, _, data_num_val, label_val, _ = data_utils.load_seg_Argo(args.filelist_val)


# data_train = np.dstack([data_train, label_train[:, :,np.newaxis]])
# data_val = np.dstack([data_val, label_val[:, :,np.newaxis]])

# depth = np.linalg.norm(data_train, 2, axis=2)
# data_train = np.concatenate([data_train, depth[:,:,np.newaxis]], axis = -1)

# depth = np.linalg.norm(data_val, 2, axis=2)
# data_val = np.concatenate([data_val, depth[:,:,np.newaxis]], axis = -1)

# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])

0it [00:00, ?it/s]

2020-01-26 09:27:16.155838-Preparing datasets...


20it [00:05,  3.92it/s]


amax [[[217.02274   84.14794  194.86005 ]
  [216.58429   59.07181  190.29214 ]
  [218.70334   82.557396 199.96799 ]
  ...
  [143.4347    13.510128  61.26613 ]
  [131.37718   10.249168 111.17418 ]
  [ 53.269745   7.663868 111.337364]]]
amin [[[-213.55579      -7.4701376  -212.08127   ]
  [-212.70149      -3.2587485  -211.15894   ]
  [-215.07878      -5.7737727  -209.11053   ]
  ...
  [ -74.75722      -0.47247314  -64.9187    ]
  [ -98.06576      -0.66090035 -173.12178   ]
  [ -82.89937      -0.5216615   -47.190056  ]]]


1it [00:00,  4.88it/s]


amax [[[215.90323   20.952639  94.15194 ]
  [155.82857   18.881943  94.48389 ]
  [162.71529   21.812824  94.183495]
  ...
  [  0.         0.         0.      ]
  [  0.         0.         0.      ]
  [  0.         0.         0.      ]]]
amin [[[-140.0768       -0.49999964 -119.15227   ]
  [-192.31703      -1.0448203  -117.7571    ]
  [-162.12273      -0.57638097 -123.3467    ]
  ...
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]]]


In [5]:
num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))

2020-01-26 09:28:52.879491-43942/1639 training/validation samples.
2020-01-26 09:28:52.879571-43942 training batches.


In [7]:
for batch_idx_train in range(batch_num):

        start_idx = (batch_size * batch_idx_train) % num_train
        end_idx = min(start_idx + batch_size, num_train)
        batch_size_train = end_idx - start_idx

        points_batch = data_train[start_idx:end_idx, ...] # data number
        points_num_batch = data_num_train[start_idx:end_idx, ...] # Number of data points 
        labels_batch = label_train[start_idx:end_idx, ...] # labels of segmentation for each point
        weights_batch = np.array(label_weights_list)[labels_batch] 

        break

In [15]:
lines = []
lines.append(
    {
        "color":'red', #x
        "vertices":[[5,0,0],[0,0,0],[0,0,0]]
    })
lines.append(
    {
        "color":'green', #y
        "vertices":[[0,0,0],[0,5,0],[0,0,0]]
    })
lines.append(
    {
        "color":'blue', #z
        "vertices":[[0,0,0],[0,0,0],[0,0,5]]
    })


In [17]:
# Vizualize pointcloud with intensity
new_pts = pd.DataFrame(points_batch.reshape(-1, 3)[:, 0:3], columns=['x', 'y', 'z'])
# new_pts["red"]  = 0
# new_pts["green"]  = 0
# new_pts["blue"]  = 0
# new_pts[["red","green", "blue"]] = classcolorlist
# new_pts.loc[list0fpointindex,["red","green", "blue"]] = blue
cloud = PyntCloud(new_pts)
cloud.plot(initial_point_size=0.02, polylines = lines)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(2.1701948642730713, 14.538530826568604, 76.5…

In [22]:
args_model = "pointcnn_seg"
args_setting = "kitti3d_x8_2048_fps"
args_repeat_num = 1

model = importlib.import_module(args_model)
setting_path = os.path.join(os.path.join(ROOT_DIR), args_model)
print(setting_path)
sys.path.append(setting_path)
setting = importlib.import_module(args_setting)

sample_num = setting.sample_num
batch_size = args_repeat_num * int ( math.ceil(max_point_num / sample_num) )
batch_size = 1
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(batch_size, None, 2), name="indices")
is_training = tf.placeholder(tf.bool, name='is_training')
pts_fts = tf.placeholder(tf.float32, shape=(batch_size, max_point_num, setting.data_dim), name='points')
global_step = tf.Variable(0, trainable=False, name='global_step')
######################################################################

# # Placeholders
# indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
# xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
# rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
# jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
# global_step = tf.Variable(0, trainable=False, name='global_step')
# is_training = tf.placeholder(tf.bool, name='is_training')

# pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
# labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
# labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')


######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if not setting.use_extra_features:
        features_sampled = None
else:
    points_sampled = pts_fts_sampled
    features_sampled = None

net = model.Net(points_sampled, features_sampled, is_training, setting)
seg_probs_op = tf.nn.softmax(net.logits, name='seg_probs')

# for restore model
saver = tf.train.Saver()

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


/home/kartik/DL_model/PointCNN/pointcnn_seg
sampling path  /home/kartik/DL_model/PointCNN/sampling
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2020-01-26 09:33:54.354001-Parameter number: 11510276.


In [25]:
import math
output_location = os.path.join(ROOT_DIR, "test_data", "results2")
with tf.Session() as sess:
    
#     sess.run(init_op)
    # Load the model
    print("here")
    latest_ckpt = tf.train.latest_checkpoint("/home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts")

    print(latest_ckpt)
    if latest_ckpt:
        print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
        saver.restore(sess, latest_ckpt)
        print('{}-Checkpoint loaded from {} (Iter {})'.format(
            datetime.now(), latest_ckpt, sess.run(global_step)))
    print("donw")
    indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))
    all_files = source_files
    # The locaton of trh classes ais defined as below
    file_count = 0
    merged_label_zero = np.zeros((label_length),dtype=int)
    merged_confidence_zero = np.zeros((label_length),dtype=float)
    
    points_batch = points_batch
    points_num_batch = data_num_train[start_idx:end_idx, ...] # Number of data points 
    labels_batch = label_train[start_idx:end_idx, ...] # labels of segmentation for each point
    weights_batch = np.array(label_weights_list)[labels_batch] 

    
    
    data =points_batch
    data_num =points_num_batch
    indices_split_to_full = indices_split_to_full[0:item_num]
    batch_num = data.shape[0]

    labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
    confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)
    print(label_length)

    #print('{}-{:d} testing batches.'.format(datetime.now(), batch_num))
    for batch_idx in tqdm(range(batch_num)):
        #if batch_idx % 10 == 0:
        #print('{}-Processing {} of {} batches.'.format(datetime.now(), batch_idx, batch_num))
        points_batch = data[[batch_idx] * batch_size, ...]
        point_num = data_num[batch_idx]
        if(point_num <= 10):
            continue 

        tile_num = int(math.ceil((sample_num * batch_size) / point_num))
        indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
        np.random.shuffle(indices_shuffle)
        indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
        indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

        seg_probs = sess.run([seg_probs_op],
                                feed_dict={
                                    pts_fts: points_batch,
                                    indices: indices_batch,
                                    is_training: False,
                                })
        probs_2d = np.reshape(seg_probs, (sample_num * batch_size, -1))

        predictions = [(-1, 0.0)] * point_num
        for idx in range(sample_num * batch_size):
            point_idx = indices_shuffle[idx]
            probs = probs_2d[idx, :]
            confidence = np.amax(probs)
            label = np.argmax(probs)
            if confidence > predictions[point_idx][1]:
                predictions[point_idx] = [label, confidence]
        labels_pred[batch_idx, 0:point_num] = np.array([label for label, _ in predictions])
        confidences_pred[batch_idx, 0:point_num] = np.array([confidence for _, confidence in predictions])



    for idx in range(batch_num): #Get highest confidence
        pred = labels_pred[idx].astype(np.int64)
        _indices = indices_split_to_full[idx].astype(np.int64)
        confidence = confidences_pred[idx].astype(np.float32)
        num = data_num[idx].astype(np.int64)

        for i in range(pred.shape[0]):

             if confidence[i] > 0.8 and confidence[i] > merged_confidence_zero[_indices[i]]:
                merged_confidence_zero[_indices[i]] = confidence[i]
                merged_label_zero[_indices[i]] = pred[i]


    all_label_pred[indices_for_prediction] = merged_label_zero
    all_label_pred.tofile(output_location+"/out_{}".format(file_count))
    file_count+=1
    print("saved to ", output_location+"/out_{}".format(file_count))
    

here
/home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946
2020-01-26 09:39:17.286214-Found checkpoint /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946
INFO:tensorflow:Restoring parameters from /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946
2020-01-26 09:39:17.406643-Checkpoint loaded from /home/kartik/DL_model/PointCNN/outfolder_argo_gr3/ckpts/iter-141946 (Iter 141946)
donw


NameError: name 'item_num' is not defined

In [9]:
args = {
        'load_ckpt' : __model,
        'repeat_num' : 1,
        'setting' : "kitti3d_x8_2048_fps",
        'save_ply' : False,
        'model' : "pointcnn_seg",
        'max_point_num' : 8192,
        'filelist' : __dataset
}

model = importlib.import_module(args['model'])
setting_path = os.path.join(_file_loc, args['model'])
sys.path.append(setting_path)
setting = importlib.import_module(args['setting'])

sample_num = setting.sample_num
max_point_num = args['max_point_num']
batch_size = args['repeat_num'] * int(math.ceil(max_point_num / sample_num))

######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(batch_size, None, 2), name="indices")
is_training = tf.placeholder(tf.bool, name='is_training')
pts_fts = tf.placeholder(tf.float32, shape=(batch_size, max_point_num, setting.data_dim), name='points')
######################################################################

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
# if setting.data_dim > 3:
#     points_sampled, features_sampled = tf.split(pts_fts_sampled,
#                                                 [3, setting.data_dim - 3],
#                                                 axis=-1,
#                                                 name='split_points_features')
#     if not setting.use_extra_features:
#         features_sampled = None
# else:
#     points_sampled = pts_fts_sampled
#     features_sampled = None

if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled

net = model.Net(points_sampled, features_sampled, is_training, setting)
seg_probs_op = tf.nn.softmax(net.logits, name='seg_probs')

# for restore model
saver = tf.train.Saver()

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


with tf.Session() as sess:
    # Load the model
#     saver.restore(sess, args['load_ckpt'])
#     print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args['load_ckpt']))
    latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
    if latest_ckpt:
        print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
        saver.restore(sess, latest_ckpt)
        print('{}-Checkpoint loaded from {} (Iter {})'.format(
            datetime.now(), latest_ckpt, sess.run(global_step)))

    

    indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))

    folder = os.path.dirname(args['filelist'])
    filenames = [os.path.join(folder, line.strip()) for line in open(args['filelist'])]
    for filename in filenames:
        print('{}-Reading {}...'.format(datetime.now(), filename))
        
#         data_h5 = h5py.File(filename)
#         data = data_h5['data'][...].astype(np.float32)
#         data_num = data_h5['data_num'][...].astype(np.int32)
#         original_label = data_h5['label'][...].astype(np.int64)
#         batch_num = data.shape[0]

        labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
        confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)

        #print('{}-{:d} testing batches.'.format(datetime.now(), batch_num))
        for batch_idx in tqdm(range(batch_num)):
            #if batch_idx % 10 == 0:
            #print('{}-Processing {} of {} batches.'.format(datetime.now(), batch_idx, batch_num))
            points_batch = data[[batch_idx] * batch_size, ...]
            point_num = data_num[batch_idx]
            if(point_num <= 10):
                continue 

            tile_num = int(math.ceil((sample_num * batch_size) / point_num))
            indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
            np.random.shuffle(indices_shuffle)
            indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
            indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

            seg_probs = sess.run([seg_probs_op],
                                    feed_dict={
                                        pts_fts: points_batch,
                                        indices: indices_batch,
                                        is_training: False,
                                    })
            probs_2d = np.reshape(seg_probs, (sample_num * batch_size, -1))

            predictions = [(-1, 0.0)] * point_num
            for idx in range(sample_num * batch_size):
                point_idx = indices_shuffle[idx]
                probs = probs_2d[idx, :]
                confidence = np.amax(probs)
                label = np.argmax(probs)
                if confidence > predictions[point_idx][1]:
                    predictions[point_idx] = [label, confidence]
            labels_pred[batch_idx, 0:point_num] = np.array([label for label, _ in predictions])
            confidences_pred[batch_idx, 0:point_num] = np.array([confidence for _, confidence in predictions])

        filename_pred = filename[:-3] + __predicate+ '_pred.h5'
        print('{}-Saving {}...'.format(datetime.now(), filename_pred))
        file = h5py.File(filename_pred, 'w')
        file.create_dataset('data_num', data=data_num)
        file.create_dataset('label_seg', data=labels_pred)
        file.create_dataset('original_label', data=original_label)
        file.create_dataset('data', data=data)
        file.create_dataset('confidence', data=confidences_pred)
        has_indices = 'indices_split_to_full' in data_h5
        if has_indices:
            file.create_dataset('indices_split_to_full', data=data_h5['indices_split_to_full'][...])
        file.close()

        if args['save_ply']:
            print('{}-Saving ply of {}...'.format(datetime.now(), filename_pred))
            filepath_label_ply = os.path.join(filename_pred[:-3] + 'ply_label')
            data_utils.save_ply_property_batch(data[:, :, 0:3], labels_pred[...],
                                               filepath_label_ply, data_num[...], setting.num_class)
        ######################################################################
    print('{}-Done!'.format(datetime.now()))


sampling path  /home/kartik/DL_model/PointCNN/sampling
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2020-01-04 10:03:13.068704-Parameter number: 11510276.


  0%|          | 0/1639 [00:00<?, ?it/s]

2020-01-04 10:03:13.573800-Reading /home/kartik/DL_model/PointCNN/data/Argo_h5_GR_scaled_depth/train/train_full_0.h5...


ValueError: Cannot feed value of shape (5, 8192, 3) for Tensor 'points:0', which has shape '(5, 10000, 4)'

In [ ]:
k

In [ ]:

pred_list = [pred for pred in os.listdir(pred_path) if pred.split(".")[-1] == "h5" and __predicate in pred]

In [ ]:
pred_list

_result_preds = {}

for i in testlists:
    _result_preds[i] = [[]] #[[]Zero, []Half, FrameID]
    
for _file in tqdm(pred_list):

    
    origin_filename =  _file.replace(__predicate+ '_pred.h5', ".h5")
    #print(os.path.join(pred_path, origin_filename))
    input_data = h5py.File(os.path.join(pred_path, origin_filename))
    label = input_data['label'][...].astype(np.int32)
    
    frameids = np.unique(label)
    
    data = h5py.File(os.path.join(pred_path, _file))
    

    data_points = data['data'][:,:,:4].astype(np.float32)
    original_label = data['original_label'][...].astype(np.int64)

        
    labels_seg = data['label_seg'][...].astype(np.int64)
    indices = data['indices_split_to_full'][...].astype(np.int64)
    confidence = data['confidence'][...].astype(np.float32)
    data_num = data['data_num'][...].astype(np.int64)
    
    for frame_id in (frameids):
        labels_seg_frame = []
        confidence_frame = []
        idx_frame = []
        ptnum_frame = []
        indices_frame = np.nonzero(label==frame_id)
        for i in  indices_frame[0]:

            ptnum_frame.append(data_num[i])
            idx_frame.append(indices[i,:data_num[i]])
            labels_seg_frame.append(labels_seg[i,:data_num[i]])
            confidence_frame.append(confidence[i,:data_num[i]])
            
        idx_frame = np.concatenate(idx_frame)
        labels_seg_frame = np.concatenate(labels_seg_frame)
        labels_seg_frame = labels_seg_frame.astype(int)
        confidence_frame = np.concatenate(confidence_frame)
        
        is_half = 1 if "_half_" in origin_filename else 0
        
        _result_preds[frame_id][is_half] = [labels_seg_frame, confidence_frame, ptnum_frame, idx_frame]
        
    

In [ ]:

final_label = {}

for _i in tqdm(testlists):
    
    if len(_result_preds[_i][0][3]) == 0: # or len(_result_preds[_i][1][3]) == 0 :
        print("NO DATA", _i)
        continue
        
    label_length = max(_result_preds[_i][0][3])+1 # max(max(_result_preds[_i][0][3]), max(_result_preds[_i][1][3]) )+1

    merged_label_zero = np.zeros((label_length),dtype=int)
    merged_confidence_zero = np.zeros((label_length),dtype=float)
    merged_label_half = np.zeros((label_length), dtype=int)
    merged_confidence_half = np.zeros((label_length), dtype=float)


    final_label[_i] =  np.zeros((label_length), dtype=int) #[[]Zero, []Half, FrameID]
    
    is_zero = True 
    for pred_data in _result_preds[_i]:
    
        labels_seg = pred_data[0].astype(np.int64)
        indices = pred_data[3].astype(np.int64)
        confidence = pred_data[1].astype(np.float32)
        data_num = np.array(pred_data[2]).astype(np.int64)

        if is_zero:
            for i in range(labels_seg.shape[0]):
                
                 if confidence[i] > merged_confidence_zero[indices[i]]:
                    merged_confidence_zero[indices[i]] = confidence[i]
                    merged_label_zero[indices[i]] = labels_seg[i]
                                
        else:
            for i in range(labels_seg.shape[0]):
                 if confidence[i] > merged_confidence_half[indices[i]]:
                    merged_confidence_half[indices[i]] = confidence[i]
                    merged_label_half[indices[i]] = labels_seg[i]
                
        is_zero = False

    final_label[_i][merged_confidence_zero >= merged_confidence_half] = merged_label_zero[merged_confidence_zero >= merged_confidence_half]
    final_label[_i][merged_confidence_zero < merged_confidence_half] = merged_label_half[merged_confidence_zero < merged_confidence_half]

In [ ]:
np.savez(__dataset_final_result + __predicate +"-final-pred.npz", final_label)

In [ ]:
__dataset_final_result + __predicate +"-final-pred.npz"